In [1]:
import csv
import os
from datetime import datetime

In [12]:
# --- 설정 ---
COMMON_CSV_PATH = './chunked_sleep_cycles.csv'

REPORT_SQL_OUTPUT_PATH = 'report_inserts.sql'
DEVICE_ENV_LOGS_SQL_OUTPUT_PATH = 'device_environment_logs_inserts.sql'

In [3]:
# 각 테이블에 필요한 CSV 컬럼명 매핑
# (DB 테이블 컬럼명 : CSV 파일 내의 컬럼명)
REPORT_CSV_COLUMN_MAP = {
    "baby_id": "baby_id",
    "start_time": "cycle_start_time",
    "end_time": "cycle_end_time",
    "duration": "actual_sleep_duration_minutes",
    "breaks_count": "breaks_count",
    "sleep_efficiency_percent": "sleep_efficiency_percent"
}

DEVICE_ENV_LOGS_CSV_COLUMN_MAP = {
    # "usage_log_id"는 CSV에 없고 NULL로 처리
    "temperature": "avg_temperature",
    "humidity": "avg_humidity",
    "brightness": "avg_brightness",
    "white_noise_level": "avg_white_noise_level",
    "recorded_at": "cycle_start_time" # sleep_mode 결정 및 recorded_at 값으로 사용
}

In [7]:
def format_sql_value(value, data_type):
    if value == "" or value is None: return "NULL"
    if data_type in ['INT', 'FLOAT']:
        try: return str(value)
        except ValueError: return "NULL"
    elif data_type in ['DATETIME', 'ENUM', 'VARCHAR']:
        processed_value = str(value).replace("'", "''")
        return f"'{processed_value}'"
    else:
        processed_value = str(value).replace("'", "''")
        return f"'{processed_value}'"

In [8]:
"""
recorded_at 시간 문자열을 기준으로 sleep_mode를 결정합니다.
00:00 ~ 05:59:59 -> night
06:00 ~ 19:59:59 -> day
20:00 ~ 23:59:59 -> night
"""


def determine_sleep_mode(record_time_str):
    if not record_time_str: return None
    try:
        dt_object = datetime.strptime(record_time_str, '%Y-%m-%d %H:%M:%S')
        hour = dt_object.hour
        if 0 <= hour < 6: return 'night'
        elif 6 <= hour < 20: return 'day'
        else: return 'night'
    except ValueError:
        print(f"경고: 날짜/시간 문자열 '{record_time_str}' 형식 오류 또는 유효하지 않은 날짜로 sleep_mode 결정 불가.")
        return None

In [9]:
def generate_inserts_from_single_csv(
    csv_filepath,
    report_sql_path, device_env_log_sql_path,
    report_col_map, device_env_col_map,
    report_db_cols_info, device_env_db_cols_info
):
    if not os.path.exists(csv_filepath):
        print(f"오류: CSV 파일 '{csv_filepath}'를 찾을 수 없습니다.")
        return

    report_inserted_count = 0
    report_skipped_count = 0
    env_log_inserted_count = 0
    env_log_skipped_count = 0

    try:
        # BOM 처리를 위해 encoding='utf-8-sig' 사용
        with open(csv_filepath, 'r', encoding='utf-8-sig', newline='') as infile, \
             open(report_sql_path, 'w', encoding='utf-8') as report_outfile, \
             open(device_env_log_sql_path, 'w', encoding='utf-8') as env_log_outfile:
            
            reader = csv.DictReader(infile)
            if not reader.fieldnames:
                print(f"오류: CSV 파일 '{csv_filepath}'에서 헤더를 읽을 수 없습니다.")
                return
            
            print(f"DEBUG: CSV DictReader가 인식한 헤더: {reader.fieldnames}")
            
            # Report 테이블 준비
            report_actual_db_col_names = [col[0] for col in report_db_cols_info]
            report_outfile.write(f"-- {os.path.basename(csv_filepath)} 파일로부터 report 테이블 INSERT 문 생성 --\n\n")

            # Device Environment Logs 테이블 준비
            env_log_actual_db_col_names = [col[0] for col in device_env_db_cols_info]
            env_log_outfile.write(f"-- {os.path.basename(csv_filepath)} 파일로부터 device_environment_logs 테이블 INSERT 문 생성 --\n\n")

            for i, row in enumerate(reader):
                # --- Report 테이블 처리 ---
                report_sql_values = {}
                report_valid_row = True
                try:
                    for db_col, csv_col in report_col_map.items():
                        target_dtype = next((dtype for name, dtype in report_db_cols_info if name == db_col), None)
                        if not target_dtype: continue # report_db_cols_info에 없는 컬럼 무시

                        if csv_col not in row:
                            print(f"경고 (report): 행 {i+1} - CSV에 '{csv_col}' (DB '{db_col}') 컬럼 없음. 이 행의 report 건너뜁니다.")
                            report_valid_row = False; break
                        
                        raw_val = row[csv_col].strip()
                        if db_col == "baby_id" and not raw_val: # 필수값 예시
                            print(f"경고 (report): 행 {i+1} - 필수 DB 컬럼 'baby_id' (CSV: '{csv_col}') 값 없음. 이 행의 report 건너뜁니다.")
                            report_valid_row = False; break

                        proc_val = raw_val
                        if raw_val:
                            if target_dtype == "INT": proc_val = int(raw_val)
                            elif target_dtype == "FLOAT": proc_val = float(raw_val)
                        report_sql_values[db_col] = format_sql_value(proc_val, target_dtype)
                    
                    if report_valid_row:
                        final_report_vals = [report_sql_values.get(name, "NULL") for name, _ in report_db_cols_info]
                        report_insert_stmt = f"INSERT INTO report ({', '.join(report_actual_db_col_names)})\n"
                        report_insert_stmt += f"VALUES ({', '.join(final_report_vals)});\n"
                        report_outfile.write(report_insert_stmt)
                        report_inserted_count += 1
                    else:
                        report_skipped_count += 1
                except ValueError as e:
                    print(f"오류 (report): 행 {i+1} - 데이터 타입 변환 오류: {e}. 원본값 관련 정보 확인 필요. 이 행의 report 건너뜁니다.")
                    report_skipped_count += 1
                except Exception as e:
                    print(f"예외 (report): 행 {i+1} - 처리 중 예외: {e}. 이 행의 report 건너뜁니다.")
                    report_skipped_count += 1


                # --- Device Environment Logs 테이블 처리 ---
                env_log_sql_values = {"usage_log_id": "NULL"} # usage_log_id는 항상 NULL
                env_log_valid_row = True
                try:
                    for db_col, csv_col in device_env_col_map.items():
                        target_dtype = next((dtype for name, dtype in device_env_db_cols_info if name == db_col), None)
                        if not target_dtype: continue

                        if csv_col not in row:
                            print(f"경고 (env_log): 행 {i+1} - CSV에 '{csv_col}' (DB '{db_col}') 컬럼 없음. 이 행의 env_log 건너뜁니다.")
                            env_log_valid_row = False; break
                        
                        raw_val = row[csv_col].strip()
                        proc_val = raw_val
                        if raw_val:
                            if target_dtype == "FLOAT": proc_val = float(raw_val)
                        env_log_sql_values[db_col] = format_sql_value(proc_val, target_dtype)

                    if not env_log_valid_row: # 이미 위에서 break 되었다면 실행 안됨
                        env_log_skipped_count += 1
                        continue # 다음 CSV 행으로

                    # sleep_mode 결정
                    recorded_at_csv_col = device_env_col_map.get("recorded_at")
                    if not recorded_at_csv_col or recorded_at_csv_col not in row:
                        print(f"경고 (env_log): 행 {i+1} - sleep_mode 결정을 위한 CSV 컬럼 ('{recorded_at_csv_col or 'recorded_at 매핑없음'}') 없음. 이 행의 env_log 건너뜁니다.")
                        env_log_skipped_count += 1; continue
                    
                    recorded_at_str = row[recorded_at_csv_col].strip()
                    sleep_mode_value = determine_sleep_mode(recorded_at_str)
                    if sleep_mode_value is None:
                        print(f"경고 (env_log): 행 {i+1} - 'sleep_mode' 값 결정 불가 (recorded_at: '{recorded_at_str}'). 이 행의 env_log 건너뜁니다.")
                        env_log_skipped_count += 1; continue
                    env_log_sql_values["sleep_mode"] = format_sql_value(sleep_mode_value, "ENUM")

                    final_env_log_vals = [env_log_sql_values.get(name, "NULL") for name, _ in device_env_db_cols_info]
                    env_log_insert_stmt = f"INSERT INTO device_environment_logs ({', '.join(env_log_actual_db_col_names)})\n"
                    env_log_insert_stmt += f"VALUES ({', '.join(final_env_log_vals)});\n"
                    env_log_outfile.write(env_log_insert_stmt)
                    env_log_inserted_count += 1

                except ValueError as e:
                    print(f"오류 (env_log): 행 {i+1} - 데이터 타입 변환 오류: {e}. 원본값 관련 정보 확인 필요. 이 행의 env_log 건너뜁니다.")
                    env_log_skipped_count += 1
                except Exception as e:
                    print(f"예외 (env_log): 행 {i+1} - 처리 중 예외: {e}. 이 행의 env_log 건너뜁니다.")
                    env_log_skipped_count += 1

        print(f"'{report_sql_path}' 파일에 report INSERT 문 {report_inserted_count}개 생성 완료. (건너뛴 행: {report_skipped_count}개)")
        print(f"'{device_env_log_sql_path}' 파일에 device_environment_logs INSERT 문 {env_log_inserted_count}개 생성 완료. (건너뛴 행: {env_log_skipped_count}개)")

    except FileNotFoundError: print(f"오류: CSV 파일 '{csv_filepath}'를 찾을 수 없습니다.")
    except Exception as e: print(f"파일 처리 중 오류 발생: {e}")

In [10]:
# --- 테이블별 DB 스키마 정보 ---
report_db_columns_schema = [
    ("baby_id", "INT"), ("start_time", "DATETIME"), ("end_time", "DATETIME"),
    ("duration", "FLOAT"), ("breaks_count", "INT"), ("sleep_efficiency_percent", "FLOAT")
]

device_env_db_columns_schema = [
    ("usage_log_id", "INT"), ("sleep_mode", "ENUM"), ("temperature", "FLOAT"),
    ("humidity", "FLOAT"), ("brightness", "FLOAT"), ("white_noise_level", "FLOAT"),
    ("recorded_at", "DATETIME")
]

In [13]:
# --- 스크립트 실행 ---
if __name__ == "__main__":
    print("--- CSV 파일에서 두 테이블 데이터 동시 생성 시작 ---")
    # MySQL DDL 변경 (report.duration FLOAT, device_environment_logs.usage_log_id NULL 허용) 선행 가정
    generate_inserts_from_single_csv(
        COMMON_CSV_PATH,
        REPORT_SQL_OUTPUT_PATH, DEVICE_ENV_LOGS_SQL_OUTPUT_PATH,
        REPORT_CSV_COLUMN_MAP, DEVICE_ENV_LOGS_CSV_COLUMN_MAP,
        report_db_columns_schema, device_env_db_columns_schema
    )
    print("--- 데이터 생성 완료 ---")

--- CSV 파일에서 두 테이블 데이터 동시 생성 시작 ---
DEBUG: CSV DictReader가 인식한 헤더: ['cycle_start_time', 'cycle_end_time', 'total_cycle_duration_minutes', 'actual_sleep_duration_minutes', 'sleep_efficiency_percent', 'breaks_count', 'avg_temperature', 'avg_humidity', 'avg_brightness', 'avg_white_noise_level', 'week', 'baby_id', 'date']
'report_inserts.sql' 파일에 report INSERT 문 29588개 생성 완료. (건너뛴 행: 0개)
'device_environment_logs_inserts.sql' 파일에 device_environment_logs INSERT 문 29588개 생성 완료. (건너뛴 행: 0개)
--- 데이터 생성 완료 ---
